In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
uri = "mongodb://localhost:27017"
db_name = "mtg_decks_db"
client = MongoClient(uri)
db = client[db_name]

collection = db["personal_pool"]

In [5]:
df = pd.read_csv("../raw-data/ligamagic.csv", encoding="ISO-8859-1")
df.head()

,Edicao (PTBR),Edicao (EN),Edicao (Sigla),Card (PT),Card (EN),Quantidade,Qualidade (M NM SP MP HP D),Idioma (BR EN DE ES FR IT JP KO RU TW),Raridade (M R U C),Cor (W U B R G M A L),Extras,Card #,Comentario
0,A Guerra dos Irmãos,The Brothers' War,bro,Desfigurar,Disfigure,1,NaN,NaN,C,B,NaN,91,NaN
1,NaN,Adventures in the Forgotten Realms,afr,Porta Secreta,Secret Door,2,SP,PTEN,C,U,NaN,71,NaN
2,NaN,Amonkhet,akh,Cartula da Solidariedade,Cartouche of Solidarity,1,NM,PTEN,C,W,NaN,7,NaN
3,Ascensão dos Eldrazi,Rise of the Eldrazi,roe,Sombra de Aranha,Spider Umbra,1,NM,NaN,C,G,NaN,208,NaN
4,NaN,Commander 2017,c17,Estepe Remota,Secluded Steppe,1,SP,PTEN,C,L,NaN,278,NaN


In [6]:
df = df[["Card (EN)", "Quantidade"]]
df = df.rename(columns={"Quantidade": "count", "Card (EN)": "name"})

df.head()

,name,count
0,Disfigure,1
1,Secret Door,2
2,Cartouche of Solidarity,1
3,Spider Umbra,1
4,Secluded Steppe,1


In [7]:
selected_columns = df[["count", "name"]]
cards = selected_columns.to_dict(orient="records")
cards

[{'count': 1, 'name': 'Disfigure'},
 {'count': 2, 'name': 'Secret Door'},
 {'count': 1, 'name': 'Cartouche of Solidarity'},
 {'count': 1, 'name': 'Spider Umbra'},
 {'count': 1, 'name': 'Secluded Steppe'},
 {'count': 4, 'name': 'Tranquil Thicket'},
 {'count': 1, 'name': 'Lead the Stampede'},
 {'count': 3, 'name': 'Negate'},
 {'count': 4, 'name': 'Palace Sentinels'},
 {'count': 1, 'name': 'Swiftwater Cliffs'},
 {'count': 1, 'name': 'Raise the Alarm'},
 {'count': 1, 'name': 'Life Goes On'},
 {'count': 2, 'name': 'Radiant Fountain'},
 {'count': 1, 'name': 'Setessan Training'},
 {'count': 1, 'name': 'Thornwood Falls'},
 {'count': 1, 'name': 'Smash to Dust'},
 {'count': 4, 'name': 'Ancient Stirrings'},
 {'count': 2, 'name': 'Divest'},
 {'count': 1, 'name': 'Sunlance'},
 {'count': 2, 'name': 'Circle of Protection: Green'},
 {'count': 1, 'name': 'Lightning Axe'},
 {'count': 3, 'name': 'Phyrexian Rager'},
 {'count': 4, 'name': 'Icatian Javelineers'},
 {'count': 1, 'name': 'Oblivion Ring'},
 {'c

In [62]:
import_type = "sum"  # sum or replace


def add_or_update_card(card):
    card_from_pool = db["pool"].find_one({"scryfall_name": card["name"]})
    name = card_from_pool["name"] if card_from_pool else card["name"]

    existing_card = collection.find_one({"scryfall_name": card["name"]})
    if not existing_card:
        collection.insert_one(
            {"name": name, "scryfall_name": card["name"], "count": card["count"]})
        return

    if import_type == "sum":
        card["count"] += existing_card["count"]

    collection.update_one({"_id": existing_card["_id"]}, {
                          "$set": {"count": card["count"]}})

In [63]:
for card in cards:
    add_or_update_card(card)